In [4]:
import pandas as pd
import numpy
from pandas import read_csv
from matplotlib import pyplot

%matplotlib inline
pd.set_option('display.width', 100)
pd.set_option('precision', 3)

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

filename = "data/boston-housing/housing.csv"

names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
dataset = read_csv(filename, names=names)

In [5]:
# Reescalar y Dividir

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split-out validation dataset
array = dataset.values
X = array[:,0:13]
Y = array[:,13]

validation_size = 0.20 #   20%
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Escalar Datos
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

In [12]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

#parametros
num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'

# Tune scaled GBM
param_grid = dict(n_estimators=numpy.array([50,100,150,200,250,300,350,400,999]))


model = GradientBoostingRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)


C:\Users\TNT\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [13]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -9.350538 using {'n_estimators': 999}
-10.812167 (4.724394) with: {'n_estimators': 50}
-10.040857 (4.441758) with: {'n_estimators': 100}
-9.694045 (4.275653) with: {'n_estimators': 150}
-9.539481 (4.270153) with: {'n_estimators': 200}
-9.449042 (4.261930) with: {'n_estimators': 250}
-9.426909 (4.271399) with: {'n_estimators': 300}
-9.366779 (4.251669) with: {'n_estimators': 350}
-9.353870 (4.265816) with: {'n_estimators': 400}
-9.350538 (4.286363) with: {'n_estimators': 999}


## Eleccion del modelo y testeo

Se elige el de 400 esstimator

In [35]:
# prepare the model y entrenarlo
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

model = GradientBoostingRegressor(random_state=seed, n_estimators=400)
model.fit(rescaledX, Y_train)



GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=400, n_iter_no_change=None, presort='auto',
             random_state=7, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

## Ahora probarlo con el 20% de los datos de validacion

In [36]:
from sklearn.metrics  import mean_squared_error


# transform the validation dataset
rescaledValidationX = scaler.transform(X_validation)

predictions = model.predict(rescaledValidationX)

print(mean_squared_error(Y_validation, predictions))

11.878916447820348


In [37]:
pd.set_option('display.width', 100)
pd.set_option('precision', 3)

for validacion, prediccion in zip(Y_validation, predictions):
    desviacion = (prediccion - validacion) * 100 / validacion
    msg = "Validacion/Prediccion %4.2f / %4.2f (%4.2f %%)" % (validacion, prediccion, desviacion)
    print (msg)

Validacion/Prediccion 21.70 / 21.60 (-0.44 %)
Validacion/Prediccion 18.50 / 18.91 (2.20 %)
Validacion/Prediccion 22.20 / 20.52 (-7.55 %)
Validacion/Prediccion 20.40 / 20.77 (1.82 %)
Validacion/Prediccion 8.80 / 4.61 (-47.59 %)
Validacion/Prediccion 8.30 / 8.99 (8.29 %)
Validacion/Prediccion 21.40 / 19.59 (-8.46 %)
Validacion/Prediccion 23.60 / 23.86 (1.09 %)
Validacion/Prediccion 23.50 / 24.78 (5.43 %)
Validacion/Prediccion 19.00 / 14.55 (-23.43 %)
Validacion/Prediccion 10.20 / 9.64 (-5.51 %)
Validacion/Prediccion 31.60 / 32.84 (3.91 %)
Validacion/Prediccion 18.40 / 16.70 (-9.24 %)
Validacion/Prediccion 19.40 / 18.27 (-5.81 %)
Validacion/Prediccion 44.00 / 40.39 (-8.20 %)
Validacion/Prediccion 23.20 / 19.23 (-17.12 %)
Validacion/Prediccion 22.00 / 25.48 (15.82 %)
Validacion/Prediccion 33.10 / 32.72 (-1.13 %)
Validacion/Prediccion 13.20 / 16.92 (28.21 %)
Validacion/Prediccion 21.70 / 20.56 (-5.27 %)
Validacion/Prediccion 16.70 / 18.15 (8.66 %)
Validacion/Prediccion 23.90 / 26.72 (11.78 